In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from herbie import Herbie
import xarray as xr
import rioxarray
import os
from pathlib import Path
import metpy.calc as mcalc
from metpy.units import units as metpy_units
from scipy.ndimage import gaussian_filter1d
from datetime import datetime, timedelta
import pyproj
from scipy.interpolate import griddata
import matplotlib.patheffects as PathEffects

"""
Uinta Basin Temperature Inversion Cross-Section Generator
- Optimized visualization with clean temperature gradients
- Selective inversion marking for better clarity
- Enhanced temperature scale with intuitive color transitions
- Higher vertical resolution up to 3800m
- Automatic collision avoidance for temperature labels
"""

# ======================================================
# CONFIGURABLE PARAMETERS
# ======================================================
# Date and time settings
ANALYSIS_DATE = "2024-03-05"  # Format: YYYY-MM-DD
ANALYSIS_HOUR = 0             # Hour in UTC (0-23)
OUTPUT_PREFIX = "uinta_basin" # Prefix for output filenames

# Uinta Basin region coordinates
BASIN_BOUNDS = {
    'lon_min': -111.0,  # Western edge
    'lon_max': -109.0,  # Eastern edge
    'lat_min': 39.5,    # Southern edge
    'lat_max': 41.0     # Northern edge
}

# Key locations to include in visualizations
LOCATIONS = {
    "Vernal": (-109.53, 40.46),
    "Roosevelt": (-110.01, 40.30),
    "Myton": (-110.06, 40.20)
}

# Additional locations (points shown but not labeled)
UNLABELED_LOCATIONS = {
    "Duchesne": (-110.40, 40.16),
    "Basin Center": (-109.80, 40.20)
}

# Pressure levels to analyze (hPa)
# NOTE: If data for certain levels can't be downloaded, the code will still work
PRESSURE_LEVELS = [850, 800, 750, 700, 650, 600]

# Surface elevation estimate (meters)
SURFACE_ELEVATION = 1500

# Temperature visualization parameters
TEMP_MIN = -20  # Minimum temperature for colormaps (°C)
TEMP_MAX = 15   # Maximum temperature for colormaps (°C)

# Cross-section line configuration - designed to pass through all three key locations
CROSS_SECTION_START = (-110.30, 40.10)
CROSS_SECTION_END = (-109.40, 40.60)

# Inversion detection parameters
MIN_INVERSION_STRENGTH = 0.5  # Minimum temperature increase (°C) to mark as inversion
INVERSION_MARKER_DENSITY = 50  # Higher = fewer markers (sample every Nth point)

# Plotting defaults
plt.rcParams["figure.figsize"] = (10, 8)
plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.size"] = 14
plt.rcParams["axes.titlesize"] = 16
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["legend.fontsize"] = 12

# Maximum height for vertical plots (meters)
MAX_HEIGHT = 3800  # Increased to 3800m

# ======================================================
def load_dem_data(dem_path=None):
    """
    Load Digital Elevation Model (DEM) data and clip it to the Uinta Basin extent.
    Downsample if necessary for performance.
    """
    if dem_path is None:
        possible_paths = [
            "uinta_basin_dem.tif",
            "merged_dem.tif",
            str(Path.home() / "PycharmProjects" / "snowshadow" / "notebooks" / "uinta_dem_data" / "merged_dem.tif"),
        ]
        for path in possible_paths:
            if os.path.exists(path):
                dem_path = path
                break

    if dem_path is None or not os.path.exists(dem_path):
        raise FileNotFoundError("Could not find DEM data file. Please specify path.")

    print(f"Loading DEM data from {dem_path}")

    # Load the DEM file with rioxarray
    dem_data = rioxarray.open_rasterio(dem_path)

    # Clip to Uinta Basin extent
    try:
        dem_data = dem_data.rio.clip_box(
            minx=BASIN_BOUNDS['lon_min'],
            miny=BASIN_BOUNDS['lat_min'],
            maxx=BASIN_BOUNDS['lon_max'],
            maxy=BASIN_BOUNDS['lat_max']
        )
    except Exception as e:
        print(f"Warning: Could not clip DEM. Error: {e}")

    # Downsample large DEMs for better visualization performance
    max_size = 500  # Maximum size for either dimension
    if dem_data.shape[1] > max_size or dem_data.shape[2] > max_size:
        scale_y = max(1, dem_data.shape[1] // max_size)
        scale_x = max(1, dem_data.shape[2] // max_size)
        scale = max(scale_x, scale_y)
        print(f"Downsampling DEM by factor of {scale} for visualization...")
        dem_data = dem_data[:, ::scale, ::scale]

    return dem_data


def download_temperature_data(date=ANALYSIS_DATE, hour=ANALYSIS_HOUR):
    """
    Download temperature data at multiple pressure levels and at the surface (2m).
    Returns a dictionary of Xarray datasets.
    """
    datetime_str = f"{date} {hour:02d}:00 UTC"
    print(f"Downloading temperature data for {datetime_str}...")

    datasets = {}

    # Get pressure-level data
    H_prs = Herbie(
        date,
        model="hrrr",
        product="prs",
        fxx=0,
        hour=hour
    )

    # Track which levels were successfully downloaded
    successful_levels = []

    for level in PRESSURE_LEVELS:
        search_query = f"TMP:{level} mb"
        try:
            ds = H_prs.xarray(search_query)
            datasets[level] = ds
            successful_levels.append(level)
            print(f"Downloaded data for {level} hPa")
        except Exception as e:
            print(f"Error downloading {level} hPa data: {e}")

    # Get surface temperature (2m)
    H_sfc = Herbie(
        date,
        model="hrrr",
        product="sfc",
        fxx=0,
        hour=hour
    )

    try:
        surface_temp = H_sfc.xarray("TMP:2 m above")
        datasets['surface'] = surface_temp
        print("Downloaded surface temperature data")
    except Exception as e:
        print(f"Error downloading surface temperature data: {e}")
        # If we can't get surface data, we need to abort
        if not datasets:
            raise ValueError("Unable to download any temperature data")

    # Print a summary of what we have
    print(f"Successfully downloaded data for surface and {len(successful_levels)} pressure levels")
    if successful_levels:
        print(f"Highest available pressure level: {min(successful_levels)} hPa")
        print(f"Lowest available pressure level: {max(successful_levels)} hPa")

    return datasets


def pressure_to_height(pressure_hPa):
    """
    Convert pressure in hPa to approximate height in meters using MetPy.
    Falls back to a scale-height approximation if needed.
    """
    try:
        pressure_val = pressure_hPa * metpy_units.hPa
        height = mcalc.pressure_to_height(pressure_val).magnitude
        return height
    except:
        # Fallback method if MetPy is not available
        scale_height = 7400  # meters
        p0 = 1013.25  # hPa (standard sea level pressure)
        return -scale_height * np.log(pressure_hPa / p0)


def get_cross_section_line(num_points=200):
    """
    Define the cross-section line by interpolating between CROSS_SECTION_START and CROSS_SECTION_END.
    Increase num_points for higher resolution.
    """
    cs_lons = np.linspace(CROSS_SECTION_START[0], CROSS_SECTION_END[0], num_points)
    cs_lats = np.linspace(CROSS_SECTION_START[1], CROSS_SECTION_END[1], num_points)
    return cs_lons, cs_lats


def create_cross_section_map(datasets, cs_lons, cs_lats, date=ANALYSIS_DATE, hour=ANALYSIS_HOUR):
    """
    Creates a map showing the location of the cross-section line.
    """
    datetime_str = f"{date} {hour:02d}:00 UTC"
    output_file = f"{OUTPUT_PREFIX}_cross_section_map_{date.replace('-', '')}_h{hour:02d}.png"

    print(f"Creating cross-section location map for {datetime_str}...")

    # Create a figure
    plt.figure(figsize=(10, 8), dpi=300)
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Add state boundaries and coastlines
    ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.8)
    ax.coastlines(resolution='50m', linewidth=0.8)

    # Add grid
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    # Plot the cross-section line
    ax.plot(cs_lons, cs_lats, 'r-', linewidth=2, transform=ccrs.PlateCarree(),
            label='Cross-Section Line')

    # Mark start and end points
    ax.plot(cs_lons[0], cs_lats[0], 'o', color='blue', markersize=8,
            transform=ccrs.PlateCarree(), label='Start')
    ax.plot(cs_lons[-1], cs_lats[-1], 's', color='green', markersize=8,
            transform=ccrs.PlateCarree(), label='End')

    # Mark key locations
    for loc_name, (lon, lat) in LOCATIONS.items():
        ax.plot(lon, lat, 'o', color='black', markersize=8, transform=ccrs.PlateCarree())
        ax.text(lon + 0.05, lat, loc_name, fontsize=12, transform=ccrs.PlateCarree(),
               bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

    # Set map extent to the Uinta Basin
    ax.set_extent([BASIN_BOUNDS['lon_min'], BASIN_BOUNDS['lon_max'],
                  BASIN_BOUNDS['lat_min'], BASIN_BOUNDS['lat_max']],
                 crs=ccrs.PlateCarree())

    # Add title and legend
    ax.set_title(f'Cross-Section Line Location - Uinta Basin\n{datetime_str}',
                fontsize=16, fontweight='bold')
    ax.legend(loc='lower right')

    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Cross-section map saved as {output_file}")
    return output_file


def create_cross_section(dem_data, datasets, date=ANALYSIS_DATE, hour=ANALYSIS_HOUR):

    datetime_str = f"{date} {hour:02d}:00 UTC"
    output_file = f"{OUTPUT_PREFIX}_cross_section_{date.replace('-', '')}_h{hour:02d}.png"

    print(f"Creating temperature cross-section plot for {datetime_str}...")

    # 1) Get cross-section line with more points for better resolution
    cs_lons, cs_lats = get_cross_section_line(num_points=300)

    # 2) Compute distances in km for the x-axis
    geod = pyproj.Geod(ellps='WGS84')
    distance_km = [0.0]
    for i in range(1, len(cs_lons)):
        _, _, dist = geod.inv(cs_lons[i-1], cs_lats[i-1], cs_lons[i], cs_lats[i])
        distance_km.append(distance_km[-1] + dist / 1000.0)
    distance_km = np.array(distance_km)

    # Prepare figure with white background
    fig = plt.figure(figsize=(14, 8), dpi=300, facecolor='white')
    ax = plt.axes(facecolor='white')  # Explicit white background

    # Extract DEM data along the cross-section
    x_coords = dem_data.x.values
    y_coords = dem_data.y.values
    elev = dem_data.values[0]

    # Extract elevations
    elevations = []
    for lon, lat in zip(cs_lons, cs_lats):
        lon_idx = np.argmin(np.abs(x_coords - lon))
        lat_idx = np.argmin(np.abs(y_coords - lat))
        elevations.append(elev[lat_idx, lon_idx])

    # Smooth terrain for better visualization
    elevations = gaussian_filter1d(elevations, sigma=2)

    # Get downloaded pressure levels (might be fewer than requested)
    available_levels = [level for level in PRESSURE_LEVELS if level in datasets]

    if not available_levels:
        print("Warning: No pressure level data available. Using only surface data.")
        # In this case, we'll create a minimal vertical grid
        heights = [3000, 3800]  # Use a couple of artificial upper levels
    else:
        heights = [pressure_to_height(p) for p in available_levels]

        # Ensure we have data up to MAX_HEIGHT
        if heights[-1] < MAX_HEIGHT:
            heights.append(MAX_HEIGHT)

    # Create points for 2D interpolation
    points = []
    temps = []

    # Add surface temperature points
    if 'surface' in datasets:
        ds_sfc = datasets['surface']
        for i, (lon, lat) in enumerate(zip(cs_lons, cs_lats)):
            # Skip points below terrain
            surface_height = max(SURFACE_ELEVATION, elevations[i])

            # Find nearest point in the grid
            dist = (ds_sfc.longitude - lon)**2 + (ds_sfc.latitude - lat)**2
            idx = dist.argmin()
            y_idx, x_idx = np.unravel_index(idx, dist.shape)

            # Get temperature in Celsius
            temp = float(ds_sfc.t2m[y_idx, x_idx].values) - 273.15

            # Add point to the dataset
            points.append([distance_km[i], surface_height])
            temps.append(temp)

    # Add pressure level temperature points
    for level in available_levels:
        level_height = pressure_to_height(level)
        ds = datasets[level]

        for i, (lon, lat) in enumerate(zip(cs_lons, cs_lats)):
            # Skip points below terrain
            if level_height <= elevations[i]:
                continue

            # Find nearest point in the grid
            dist = (ds.longitude - lon)**2 + (ds.latitude - lat)**2
            idx = dist.argmin()
            y_idx, x_idx = np.unravel_index(idx, dist.shape)

            # Get temperature in Celsius
            temp = float(ds.t[y_idx, x_idx].values) - 273.15

            # Add point to the dataset
            points.append([distance_km[i], level_height])
            temps.append(temp)

    # Convert to numpy arrays for interpolation
    points = np.array(points)
    temps = np.array(temps)

    # Create a more dense grid for visualization
    x_grid = np.linspace(min(distance_km), max(distance_km), 500)
    y_grid = np.linspace(SURFACE_ELEVATION, MAX_HEIGHT, 500)
    xx, yy = np.meshgrid(x_grid, y_grid)

    # Use cubic interpolation for smoother results
    grid_z = griddata(points, temps, (xx, yy), method='cubic')

    # For any NaN values (particularly at edges), fill with nearest neighbor method
    grid_z_nn = griddata(points, temps, (xx, yy), method='nearest')
    mask = np.isnan(grid_z)
    grid_z[mask] = grid_z_nn[mask]

    # Apply some light smoothing to reduce interpolation artifacts
    from scipy.ndimage import gaussian_filter
    grid_z = gaussian_filter(grid_z, sigma=1.0)

    # Create a mask for below-terrain areas
    terrain_mask = np.zeros_like(grid_z, dtype=bool)

    # Interpolate terrain to the same x-grid
    terrain_heights = np.interp(x_grid, distance_km, elevations)

    # Create the mask
    for i, x in enumerate(x_grid):
        terrain_height = terrain_heights[i]
        for j, y in enumerate(y_grid):
            if y < terrain_height:
                terrain_mask[j, i] = True

    # Apply the mask
    grid_z_masked = np.ma.array(grid_z, mask=terrain_mask)

    # Create a custom colormap that emphasizes the temperature range
    from matplotlib.colors import LinearSegmentedColormap

    # Create a custom diverging colormap with a clear transition at 0°C
    colors = [(0, 0, 0.6),    # dark blue for coldest
              (0.2, 0.4, 0.8), # blue
              (0.4, 0.7, 0.9), # light blue
              (0.7, 0.9, 1.0), # very light blue
              (1.0, 1.0, 1.0), # white for zero
              (1.0, 0.9, 0.4), # light yellow
              (1.0, 0.7, 0.0), # yellow
              (0.9, 0.4, 0.0), # orange
              (0.7, 0.0, 0.0)] # dark red for warmest

    cmap_name = 'custom_temp'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=256)

    # Create a custom normalization centered on 0°C
    norm = mcolors.TwoSlopeNorm(vmin=TEMP_MIN, vcenter=0, vmax=TEMP_MAX)

    # Draw the contour plot with masked data
    contour = ax.pcolormesh(xx, yy, grid_z_masked,
                           cmap=cm,
                           norm=norm,
                           shading='gouraud')  # Smooth shading

    # Add contour lines at specific intervals
    contour_lines = ax.contour(xx, yy, grid_z_masked,
                              levels=np.arange(TEMP_MIN, TEMP_MAX+1, 5),
                              colors='black',
                              linewidths=0.7,
                              alpha=0.7)

    # Add contour labels with white background and collision detection
    # Set use_clabeltext=True for more control over label placement
    contour_labels = ax.clabel(
        contour_lines,
        inline=True,
        fontsize=10,
        fmt='%d°C',
        use_clabeltext=True,  # Important for collision detection
        colors='black',
        # Reduce density of labels to avoid overcrowding
        inline_spacing=15,    # Increase spacing between labels
        manual=False          # Let matplotlib handle initial placement
    )

    # Apply white background to labels and check for overlaps
    label_positions = []
    final_labels = []

    for label in contour_labels:
        # Get label position
        x, y = label.get_position()

        # Check for overlap with existing labels
        overlap = False
        for lx, ly in label_positions:
            # Simple distance-based overlap detection
            if np.sqrt((x - lx)**2 + (y - ly)**2) < 40:  # Adjust threshold as needed
                overlap = True
                break

        if not overlap:
            # Keep this label
            label_positions.append((x, y))
            final_labels.append(label)

            # Add white background with black edge
            label.set_bbox(dict(facecolor='white', edgecolor='black', alpha=0.7, pad=2))
            # Add slight offset to text to avoid touching the contour line
            label.set_horizontalalignment('center')
            label.set_verticalalignment('center')
        else:
            # Remove overlapping label
            label.remove()

    # Add colorbar
    cbar = fig.colorbar(contour, ax=ax, pad=0.02, aspect=30)
    cbar.set_label('Temperature (°C)', fontsize=14)

    # Plot terrain with a cleaner appearance
    ax.fill_between(x_grid, SURFACE_ELEVATION, terrain_heights,
                   color='saddlebrown', alpha=0.8,
                   edgecolor='black', linewidth=1.0)

    # Detect temperature inversions (where temperature increases with height)
    inversion_x = []
    inversion_y = []

    # Look for significant inversions (sampling fewer points for clarity)
    for i in range(0, xx.shape[1], INVERSION_MARKER_DENSITY):  # Sample fewer points
        # Get the column of temperature data
        x_pos = xx[0, i]
        column = grid_z_masked[:, i]
        heights = yy[:, i]

        # Previous temperature value
        prev_temp = None

        # Loop through the column (starting above terrain)
        for j in range(0, len(column)-1, 10):  # Check every 10th point
            # Skip masked points (below terrain) and NaN values
            if terrain_mask[j, i] or np.isnan(column[j]):
                continue

            if prev_temp is not None:
                # If temperature increases with height by at least MIN_INVERSION_STRENGTH
                temp_change = column[j] - prev_temp
                if temp_change >= MIN_INVERSION_STRENGTH:
                    inversion_x.append(x_pos)
                    inversion_y.append(heights[j])

            # Update previous temperature
            prev_temp = column[j]

    # Plot inversion markers
    if inversion_x:  # Only if inversions were detected
        ax.plot(inversion_x, inversion_y, 'x', color='yellow', markersize=7,
               markeredgewidth=1.5, alpha=0.9, zorder=10)

        # Add explanation text for inversion markers
        ax.text(0.02, 0.98, "Yellow X marks = Inversions",
               transform=ax.transAxes,
               fontsize=12,
               fontweight='bold',
               color='black',
               va='top',
               bbox=dict(facecolor='white', edgecolor='black',
                        alpha=0.8, boxstyle='round,pad=0.2'))

    # Mark key locations directly on the terrain (not floating above)
    all_locations = {**LOCATIONS, **UNLABELED_LOCATIONS}
    for loc_name, (loc_lon, loc_lat) in all_locations.items():
        dists = np.sqrt((cs_lons - loc_lon)**2 + (cs_lats - loc_lat)**2)
        nearest_idx = np.argmin(dists)

        if dists[nearest_idx] < 0.15:  # within 0.15 degrees
            # Get the x position
            x_pos = distance_km[nearest_idx]

            # Position label directly on the terrain
            # First, interpolate terrain height at this position
            terrain_idx = np.argmin(np.abs(x_grid - x_pos))

            # Place marker on terrain
            y_pos = terrain_heights[terrain_idx]

            # Add vertical dashed line for clarity
            ax.axvline(x=x_pos, color='white', linestyle='--', alpha=0.5, linewidth=0.75, zorder=7)

            if loc_name in LOCATIONS:  # label only key locations
                # Create text with contrasting outline effect to ensure visibility against any background
                if x_pos < 30 or x_pos > 70:  # Adjust position based on where in the plot
                    ha = 'center'  # Central alignment for ends
                else:
                    ha = 'center'  # Center alignment for middle

                # Create text with contrasting outline
                text = ax.text(
                    x_pos, y_pos - 25,  # Position slightly below terrain
                    loc_name,
                    color='white',
                    fontsize=10,
                    fontweight='bold',
                    ha=ha,
                    va='top',
                    zorder=8,
                    bbox=dict(
                        facecolor='black',
                        alpha=0.7,
                        boxstyle='round,pad=0.2',
                        edgecolor='white',
                        linewidth=0.5
                    )
                )

                # Add path effects for better visibility
                text.set_path_effects([
                    PathEffects.withStroke(linewidth=2, foreground='black')
                ])

    # Axis labels and title
    ax.set_xlabel('Distance Along Cross-Section (km)', fontsize=14)
    ax.set_ylabel('Height (m)', fontsize=14)
    ax.set_title(f'Temperature Cross-Section - Uinta Basin\n{datetime_str}',
                fontsize=16, fontweight='bold')

    # Set y-limit to start at SURFACE_ELEVATION and end at MAX_HEIGHT
    ax.set_ylim(SURFACE_ELEVATION, MAX_HEIGHT)

    # Turn off grid
    ax.grid(False)

    # Save and close
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Cross-section plot saved as {output_file}")
    return output_file


def analyze_multiple_times(start_date, end_date, hours=None):
    """
    Generate cross-sections for multiple dates and times in one go.
    Default increments of 6 hours if hours is None.
    """
    if hours is None:
        hours = [0, 6, 12, 18]

    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    dem_data = load_dem_data()

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime("%Y-%m-%d")
        for hr in hours:
            print(f"\n==== Processing {date_str} {hr:02d}:00 UTC ====")
            try:
                datasets = download_temperature_data(date_str, hr)
                create_cross_section(dem_data, datasets, date_str, hr)
            except Exception as e:
                print(f"Error processing {date_str} {hr:02d}:00 UTC: {e}")
                print("Skipping to next time step")
        current_dt += timedelta(days=1)

    print("\nAll cross-sections created successfully!")


def main():
    """
    Main execution function.
    Downloads data for ANALYSIS_DATE/ANALYSIS_HOUR, creates cross-section.

    Uncomment the analyze_multiple_times call to process a range of dates.
    """
    # Load DEM data once to reuse
    dem_data = load_dem_data()

    # Get temperature data
    datasets = download_temperature_data(ANALYSIS_DATE, ANALYSIS_HOUR)

    # Create cross-section visualization
    create_cross_section(dem_data, datasets, ANALYSIS_DATE, ANALYSIS_HOUR)

    # Uncomment to analyze a range of dates/times
    # analyze_multiple_times("2023-02-01", "2023-02-07", [0, 12])

    print("Cross-section analysis complete!")


if __name__ == "__main__":
    main()

Loading DEM data from /Users/a02428741/PycharmProjects/snowshadow/notebooks/uinta_dem_data/merged_dem.tif
Downsampling DEM by factor of 14 for visualization...
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Mar-05 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
Downloaded data for 850 hPa
Downloaded data for 800 hPa
Downloaded data for 750 hPa
Downloaded data for 700 hPa
Downloaded data for 650 hPa
Downloaded data for 600 hPa
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-05 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
Downloaded surface temperature data
Successfully downloaded data for surface and 6 pressure levels
Highest available pressure level: 600 hPa
Lowest available pressure level: 850 hPa
Creating temperature cross-section plot for 2024-03-05 00:00 UTC...
Cross-section plot saved as uinta_basin_cross_section_20240305_h00.png
Loading DEM data from /Users/a02428741/PycharmProjects/snowshadow/notebooks/uinta_dem_data/merged_dem.tif
Downsampling DEM by factor of 14 for visualization...

==== Process